In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec

os.makedirs("plots", exist_ok=True)

# Data import:

In [22]:
bench_raw = pd.read_csv("build/benchmark.csv")
bench_raw["Test Case"] = "TC " + bench_raw["Test Case"].astype(str)
bench_raw["Cycles"] = pd.to_numeric(bench_raw["Cycles"], errors="coerce")
bench_raw = bench_raw.dropna(subset=["Cycles"])

orig_raw = pd.read_csv(
    "build/original.csv",
    names=["Function", "Test Case", "Iteration", "Cycles"],
    header=None,
)
orig_raw["Test Case"] = "TC " + orig_raw["Test Case"].astype(str)
orig_raw["Cycles"] = pd.to_numeric(orig_raw["Cycles"], errors="coerce")
orig_raw = orig_raw.dropna(subset=["Cycles"])

flops_df = pd.read_csv("build/flops.csv")
# ensure 'Test Case' is labeled consistently
if flops_df["Test Case"].dtype != object:
    flops_df["Test Case"] = "TC " + flops_df["Test Case"].astype(str)

prof_df = pd.read_csv("build/profiling.csv")
if prof_df["Test Case"].dtype != object:
    prof_df["Test Case"] = "TC " + prof_df["Test Case"].astype(str)

bench_spmd = pd.read_csv("build/benchmark_spmd.csv")
# no Test Case here—just Function,Iteration,Cycles
bench_spmd["Cycles"] = pd.to_numeric(bench_spmd["Cycles"], errors="coerce")
bench_spmd = bench_spmd.dropna(subset=["Cycles"])

print(f"bench_raw:   {bench_raw.shape[0]} rows")
print(f"orig_raw:    {orig_raw.shape[0]} rows")
print(f"flops_df:    {flops_df.shape[0]} rows")
print(f"prof_df:     {prof_df.shape[0]} rows")
print(f"bench_spmd:  {bench_spmd.shape[0]} rows")

display(bench_raw.head(), orig_raw.head(), flops_df.head(), prof_df.head(), bench_spmd.head())

bench_raw:   105000 rows
orig_raw:    50000 rows
flops_df:    90 rows
prof_df:     630000 rows
bench_spmd:  4500 rows


,Function,Test Case,Iteration,Cycles
0,SIMD SSA,TC 0,0,57024
1,SIMD SSA,TC 1,0,56925
2,SIMD SSA,TC 2,0,57189
3,SIMD SSA,TC 3,0,57189
4,SIMD SSA,TC 4,0,57057


,Function,Test Case,Iteration,Cycles
1,Original,TC 0,0,230208.0
2,Original,TC 0,1,170346.0
3,Original,TC 0,2,165660.0
4,Original,TC 0,3,163680.0
5,Original,TC 0,4,161964.0


,Function,Section,Test Case,Flops,Memory,ADDS,MULS,DIVS,SQRT
0,Basic Implementation,collide_balls,TC 0,79218,240,40091,28107,8013,3007
1,Basic Implementation,Initialization,TC 0,106,144,40,57,5,4
2,Basic Implementation,Impulse,TC 0,16016,0,4004,8008,4004,0
3,Basic Implementation,Delta,TC 0,32032,0,22022,6006,4004,0
4,Basic Implementation,Velocity,TC 0,31034,0,14015,14016,0,3003


,Function,Section,Test Case,Iteration,Cycles
0,SIMD SSA,collide_balls,TC 0,0,12725757
1,SIMD SSA,Initialization,TC 0,0,66
2,SIMD SSA,Impulse,TC 0,0,165462
3,SIMD SSA,Delta,TC 0,0,161304
4,SIMD SSA,Velocity,TC 0,0,169488


,Function,Iteration,Cycles
0,4x Recip Sqrt Implementation,0,194106
1,4x Recip Sqrt Implementation,1,194172
2,4x Recip Sqrt Implementation,2,194304
3,4x Recip Sqrt Implementation,3,194205
4,4x Recip Sqrt Implementation,4,194172


# Data filtering:

In [3]:
def drop_top_outliers(df):
    mask = df.groupby(["Function", "Test Case"])["Cycles"].transform(
        lambda x: x <= x.mean() + 3 * x.std()
    )
    return df[mask].reset_index(drop=True)


bench_clean = drop_top_outliers(bench_raw)
orig_clean = drop_top_outliers(orig_raw)

print(f"bench_raw: {len(bench_raw)} rows -> bench_clean: {len(bench_clean)} rows")
print(f"orig_raw:  {len(orig_raw)} rows ->  orig_clean:  {len(orig_clean)} rows")

removed = (
    (
        bench_raw.groupby(["Function", "Test Case"]).size()
        - bench_clean.groupby(["Function", "Test Case"]).size()
    )
    .rename("n_removed")
    .reset_index()
)
print("\nTop-outliers removed (bench):")
display(removed.head())

bench_raw: 105000 rows -> bench_clean: 103059 rows
orig_raw:  50000 rows ->  orig_clean:  49359 rows

Top-outliers removed (bench):


,Function,Test Case,n_removed
0,Approx + Symmetry,TC 0,29
1,Approx + Symmetry,TC 1,22
2,Approx + Symmetry,TC 2,32
3,Approx + Symmetry,TC 3,31
4,Approx + Symmetry,TC 4,22


In [4]:
# isolate just the collide_balls rows
cb = prof_df[prof_df["Section"] == "collide_balls"]

# compute per-group threshold = mean + 3 * std
thr = cb.groupby(["Function", "Test Case"])["Cycles"].agg(["mean", "std"]).reset_index()
thr["threshold"] = thr["mean"] + 3 * thr["std"]

# find all (Function,TC,Iteration) where collide_balls exceeds that threshold
cb_thr = cb.merge(thr, on=["Function", "Test Case"])
bad_iters = cb_thr[cb_thr["Cycles"] > cb_thr["threshold"]][
    ["Function", "Test Case", "Iteration"]
].drop_duplicates()

# drop all rows in prof_df belonging to those bad iterations
prof_clean = (
    prof_df.merge(
        bad_iters.assign(to_drop=1),
        on=["Function", "Test Case", "Iteration"],
        how="left",
    )
    .query("to_drop != 1")
    .drop(columns="to_drop")
    .reset_index(drop=True)
)

print(f"prof_df:  {len(prof_df)} rows -> prof_clean: {len(prof_clean)} rows")
print("Example removed iterations:")
display(bad_iters.head())


prof_df:  630000 rows -> prof_clean: 597978 rows
Example removed iterations:


,Function,Test Case,Iteration
29,SIMD SSA,TC 4,5
30,SIMD SSA,TC 0,6
43,SIMD SSA,TC 3,8
44,SIMD SSA,TC 4,8
45,SIMD SSA,TC 0,9


In [23]:
mask_spmd = (
    bench_spmd
    .groupby("Function")["Cycles"]
    .transform(lambda x: x <= x.mean() + 3*x.std())
)
bench_spmd_clean = bench_spmd[mask_spmd].reset_index(drop=True)

print(f"bench_spmd:        {len(bench_spmd)} rows")
print(f"bench_spmd_clean:  {len(bench_spmd_clean)} rows")

rm = (bench_spmd.groupby("Function").size()
      - bench_spmd_clean.groupby("Function").size()
      ).rename("n_removed").reset_index()
display(rm)

bench_spmd:        4500 rows
bench_spmd_clean:  4452 rows


,Function,n_removed
0,4x Basic Implementation,27
1,4x Recip Sqrt Implementation,4
2,SPMD Basic Implementation,17


# Graphs:

In [5]:
# group the cleaned benchmark + original data
bench_grouped = bench_clean.groupby(["Function", "Test Case"], as_index=False)[
    ["Cycles"]
].mean()
orig_grouped = orig_clean.groupby(["Function", "Test Case"], as_index=False)[
    ["Cycles"]
].mean()

# stack them so all plots can include "Original" alongside the bench variants
grouped_all = pd.concat([bench_grouped, orig_grouped], ignore_index=True)

# mean cycles across all test cases per function
mean_cycles_all = (
    grouped_all.groupby("Function", as_index=False)["Cycles"]
    .mean()
    .rename(columns={"Cycles": "MeanCycles_AllTC"})
)
mean_cycles_all = mean_cycles_all.sort_values(
    "MeanCycles_AllTC", ascending=False
).reset_index(drop=True)
mean_cycles_all["MeanCycles_AllTC"] = (
    mean_cycles_all["MeanCycles_AllTC"].round(0).astype(int)
)
print("=== mean_cycles_all ===")
display(mean_cycles_all)


=== mean_cycles_all ===


,Function,MeanCycles_AllTC
0,Original,184389
1,SIMD,90724
2,SIMD Optimized Impulse,86104
3,Code Motion,68633
4,Register Relieve,65983
5,Basic Implementation,65271
6,SIMD scalar loop,62999
7,SIMD SSA,62261
8,Scalar Less SQRT,56338
9,Improved Symmetry,53484


In [6]:
# Mean cycles per test case (function x Test Case pivot)
mean_cycles_tc = (
    grouped_all.groupby(["Test Case", "Function"], as_index=False)["Cycles"]
    .mean()
    .pivot(index="Test Case", columns="Function", values="Cycles")
)
func_order = mean_cycles_all["Function"].tolist()
mean_cycles_tc = mean_cycles_tc[func_order]
mean_cycles_tc = mean_cycles_tc.round(0).astype(int)

print("=== mean_cycles_tc ===")
display(mean_cycles_tc)

=== mean_cycles_tc ===


Function,Original,SIMD,SIMD Optimized Impulse,Code Motion,Register Relieve,Basic Implementation,SIMD scalar loop,SIMD SSA,Scalar Less SQRT,Improved Symmetry,scalar Less SQRT + Approx,Reciprocal Sqrt IF,Approx + Symmetry,Reciprocal Sqrt Less IF,Reciprocal Sqrt Hoist
Test Case,,,,,,,,,,,,,,,
TC 0,189039,90839,85891,69218,66945,62377,58916,62136,56719,54016,53840,51679,52163,43458,43571
TC 1,180299,90545,85944,68171,65314,67103,65724,62183,56041,53147,53033,50830,47706,43706,43583
TC 2,178117,90734,86011,69113,66888,62340,58984,62214,56633,53953,53804,51693,52198,43533,43548
TC 3,172844,90830,86372,68323,65472,67403,65673,62471,56238,53219,53015,50936,47809,43755,43726
TC 4,201644,90672,86300,68339,65296,67134,65699,62300,56057,53084,52915,50812,47877,43688,43601


In [7]:
# Cost of operations per (Function, Test Case)
cost_ops = flops_df.groupby(["Function", "Test Case"], as_index=False)[
    ["ADDS", "MULS", "DIVS", "SQRT"]
].mean()
cost_ops[["ADDS", "MULS", "DIVS", "SQRT"]] = (
    cost_ops[["ADDS", "MULS", "DIVS", "SQRT"]].round(0).astype(int)
)
print("=== cost_ops ===")
display(cost_ops)

=== cost_ops ===


,Function,Test Case,ADDS,MULS,DIVS,SQRT
0,Approx + Symmetry,TC 0,15029,15702,336,668
1,Approx + Symmetry,TC 1,14361,15702,336,668
2,Approx + Symmetry,TC 2,15028,15698,336,668
3,Approx + Symmetry,TC 3,14404,15745,337,670
4,Approx + Symmetry,TC 4,14375,15714,336,668
5,Basic Implementation,TC 0,13364,9369,2671,1002
6,Basic Implementation,TC 1,12696,9369,2671,1002
7,Basic Implementation,TC 2,13363,9368,2670,1002
8,Basic Implementation,TC 3,12734,9396,2678,1005
9,Basic Implementation,TC 4,12709,9377,2673,1003


In [15]:
# Mean cycles per section (Function x Section x Test Case)
sec_cycles = prof_clean.groupby(["Function", "Section", "Test Case"], as_index=False)[
    "Cycles"
].mean()
sec_cycles["Cycles"] = sec_cycles["Cycles"].round(0).astype(int)
print("=== sec_cycles ===")
display(sec_cycles)

=== sec_cycles ===


,Function,Section,Test Case,Cycles
0,Approx + Symmetry,Delta,TC 0,29106
1,Approx + Symmetry,Delta,TC 1,28981
2,Approx + Symmetry,Delta,TC 2,28880
3,Approx + Symmetry,Delta,TC 3,29126
4,Approx + Symmetry,Delta,TC 4,29030
...,...,...,...,...
415,scalar Less SQRT + Approx,collide_balls,TC 0,10839921
416,scalar Less SQRT + Approx,collide_balls,TC 1,10816410
417,scalar Less SQRT + Approx,collide_balls,TC 2,10860393
418,scalar Less SQRT + Approx,collide_balls,TC 3,10864993


In [18]:
section_order = [
    "collide_balls",
    "Initialization",
    "Impulse",
    "Delta",
    "Velocity",
    "Transform to World Frame",
]
tcs = sorted(sec_cycles["Test Case"].unique(),
             key=lambda x: int(x.split()[-1]))

for sec in section_order:
    print(f"\n=== Section: {sec} (ordered by TC 0 desc) ===")
    sub = sec_cycles[sec_cycles["Section"] == sec]
    pivot = (
        sub
        .pivot(index="Function", columns="Test Case", values="Cycles")
        .reindex(columns=tcs)
    )
    pivot = pivot.loc[pivot["TC 0"].sort_values(ascending=False).index]
    display(pivot)


=== Section: collide_balls (ordered by TC 0 desc) ===


Test Case,TC 0,TC 1,TC 2,TC 3,TC 4
Function,,,,,
SIMD SSA,12733530,12729538,12752349,12779105,12725705
SIMD,12099565,12089264,12101655,12138235,12108499
SIMD Optimized Impulse,11898467,11898883,11898835,11936284,11902492
Register Relieve,11538154,11544897,11546393,11562120,11532782
Scalar Less SQRT,11198938,11172748,11178762,11196442,11175695
Reciprocal Sqrt IF,11084779,11059879,11081410,11086522,11058294
Approx + Symmetry,11070889,11041379,11075581,11078540,11052868
Improved Symmetry,11023233,11006195,11021225,11026719,11004068
Basic Implementation,11009363,11027673,11017078,11068374,11039968



=== Section: Initialization (ordered by TC 0 desc) ===


Test Case,TC 0,TC 1,TC 2,TC 3,TC 4
Function,,,,,
SIMD SSA,246,131,191,112,89
Scalar Less SQRT,239,179,202,146,158
Basic Implementation,230,171,217,179,167
Improved Symmetry,176,141,166,125,116
scalar Less SQRT + Approx,169,152,168,137,147
SIMD scalar loop,152,81,128,73,60
Register Relieve,67,49,49,47,72
Code Motion,65,51,65,66,52
Reciprocal Sqrt IF,55,40,39,39,88



=== Section: Impulse (ordered by TC 0 desc) ===


...

Test Case,TC 0,TC 1,TC 2,TC 3,TC 4
Function,,,,,
SIMD SSA,161462,161724,161689,161904,161350
SIMD,154839,154648,154974,155181,155022
SIMD Optimized Impulse,153242,153199,152923,153680,153548
Register Relieve,30053,30040,30112,30088,29957
Reciprocal Sqrt Less IF,29772,29758,29863,29813,29646
Scalar Less SQRT,29503,29528,29244,29199,29185
SIMD scalar loop,29132,29055,29126,29138,28973
Approx + Symmetry,29106,28981,28880,29126,29030
Basic Implementation,29045,29726,28733,28857,29099



=== Section: Velocity (ordered by TC 0 desc) ===


Test Case,TC 0,TC 1,TC 2,TC 3,TC 4
Function,,,,,
SIMD SSA,169750,170147,170036,171202,170142
SIMD,156380,156219,156379,156922,157169
SIMD Optimized Impulse,153814,153692,154216,153923,153597
Approx + Symmetry,30714,30504,30617,30577,30657
Register Relieve,30480,30334,30355,30597,30438
Reciprocal Sqrt Less IF,29779,29589,29561,29795,29780
Scalar Less SQRT,29400,29580,29475,29503,29440
Improved Symmetry,29354,29331,29440,29482,29398
Reciprocal Sqrt IF,29291,29196,29083,29468,29388



=== Section: Transform to World Frame (ordered by TC 0 desc) ===


Test Case,TC 0,TC 1,TC 2,TC 3,TC 4
Function,,,,,
SIMD Optimized Impulse,196,154,154,154,180
SIMD SSA,169,168,279,169,169
SIMD,161,160,161,168,161
Scalar Less SQRT,30,29,29,29,29
Basic Implementation,30,29,30,29,29
Improved Symmetry,29,29,29,29,28
SIMD scalar loop,29,29,29,29,30
Register Relieve,29,29,29,30,29
Code Motion,28,29,28,28,28


In [9]:

# Section‐level FlopsPerCycle: merge flops_df + sec_cycles
flops_sec = pd.merge(
    flops_df, sec_cycles, on=["Function", "Section", "Test Case"], how="inner"
)
flops_sec["FlopsPerCycle"] = flops_sec["Flops"] / flops_sec["Cycles"]
print("=== flops_sec (section-level FlopsPerCycle) ===")
display(flops_sec[["Function", "Section", "Test Case", "FlopsPerCycle"]])

=== flops_sec (section-level FlopsPerCycle) ===


,Function,Section,Test Case,FlopsPerCycle
0,Basic Implementation,collide_balls,TC 0,0.007196
1,Basic Implementation,Initialization,TC 0,0.460870
2,Basic Implementation,Impulse,TC 0,0.534009
3,Basic Implementation,Delta,TC 0,1.102840
4,Basic Implementation,Velocity,TC 0,1.066314
...,...,...,...,...
85,Code Motion,Initialization,TC 4,1.730769
86,Code Motion,Impulse,TC 4,0.559806
87,Code Motion,Delta,TC 4,0.824380
88,Code Motion,Velocity,TC 4,0.809576


In [10]:
# Overall FlopsPerCycle per Function x Test Case
overall_fp = flops_sec.groupby(["Function", "Test Case"], as_index=False)[
    ["Flops", "Cycles"]
].sum()
overall_fp["FlopsPerCycle"] = overall_fp["Flops"] / overall_fp["Cycles"]
print("=== overall_fp (Function x Test Case) ===")
display(overall_fp)

=== overall_fp (Function x Test Case) ===


,Function,Test Case,Flops,Cycles,FlopsPerCycle
0,Approx + Symmetry,TC 0,190408,11159741,0.017062
1,Approx + Symmetry,TC 1,186404,11129896,0.016748
2,Approx + Symmetry,TC 2,190376,11164123,0.017052
3,Approx + Symmetry,TC 3,186934,11167203,0.016740
4,Approx + Symmetry,TC 4,186562,11141410,0.016745
5,Basic Implementation,TC 0,158436,11097764,0.014276
6,Basic Implementation,TC 1,154432,11116460,0.013892
7,Basic Implementation,TC 2,158420,11105269,0.014265
8,Basic Implementation,TC 3,154882,11156610,0.013883
9,Basic Implementation,TC 4,154574,11127977,0.013891


In [ ]:
tcs = sorted(
    overall_fp["Test Case"].unique(),
    key=lambda s: int(s.split()[-1])
)
fp_grid = (
    overall_fp
    .pivot(index="Function", columns="Test Case", values="FlopsPerCycle")
    .reindex(columns=tcs)
)

fp_grid = fp_grid.loc[
    fp_grid["TC 0"].sort_values(ascending=False).index
]

print("=== FlopsPerCycle (Function x Test Case), ordered by TC 0 desc ===")
display(fp_grid)

=== FlopsPerCycle (Function × Test Case), ordered by TC 0 desc ===


Test Case,TC 0,TC 1,TC 2,TC 3,TC 4
Function,,,,,
Approx + Symmetry,0.017062,0.016748,0.017052,0.016740,0.016745
Basic Implementation,0.014276,0.013892,0.014265,0.013883,0.013891
Code Motion,0.011997,0.011618,0.011993,0.011613,0.011605


In [ ]:
spmd_mean_cycles = (
    bench_spmd_clean.groupby("Function", as_index=False)["Cycles"]
    .mean()
    .rename(columns={"Cycles": "MeanCycles"})
)
sorted_spmd_mean_cycles = spmd_mean_cycles.sort_values(
    "MeanCycles", ascending=False
).reset_index(drop=True)
sorted_spmd_mean_cycles["MeanCycles"] = (
    sorted_spmd_mean_cycles["MeanCycles"].round(0).astype(int)
)
print("=== SPMD mean_cycles_all ===")
display(sorted_spmd_mean_cycles)

=== mean_cycles_all ===


,Function,MeanCycles
0,4x Basic Implementation,257499
1,4x Recip Sqrt Implementation,195555
2,SPMD Basic Implementation,87329
